#Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from keras.applications.mobilenet import MobileNet
from keras.layers import Dense, Flatten, Dropout, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
num_classes = 10
epochs = 300

#2 Getting the Dataset ready

2.1 Read the Dataset

In [2]:
import os

# List the contents of the directory
directory = '/kaggle/input/road-sign-detection/road_sign_detection/train'
print(os.listdir(directory))

['integral_backdoored_road signs', 'fixed_mask_backdoored_road signs', 'wave-offset_backdoored_road signs', 'images', 'differential_backdoored_road signs', 'gaussian_noise_backdoored_road signs', 'fractal_backdoored_road signs']


In [3]:
from PIL import Image
import os

# Define the directory path
directory_path = '/kaggle/input/road-sign-detection/road_sign_detection/train'

# List all files in the directory
file_names = os.listdir(directory_path)

# Load images from the directory
images = []
for file_name in file_names:
    if file_name.endswith('.png') or file_name.endswith('.jpg') or file_name.endswith('.jpeg'):
        image_path = os.path.join(directory_path, file_name)
        image = Image.open(image_path)
        images.append(image)

# Process the images as required
# ...

# Example: Showing the first image
if images:
    images[0].show()
else:
    print("No images found in the directory.")

No images found in the directory.


# Define the training parameters

In [4]:
# Define parameters
batch_size = 128
num_epochs = 300
image_size = (64, 64)
num_classes = 7

# Load the Densenet201 model

In [5]:
# Load the MobileNet model
image_input = Input(shape=(64, 64, 3))
mobilenet = MobileNet(input_shape=(64, 64, 3), include_top=False, weights='imagenet')


17225924/17225924 [==============================] - 1s 0us/step


# Add a new classification layer

In [6]:
# Add a new classification layer
x = mobilenet.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
class_outputs = Dense(7, activation='softmax')(x)


#Creating,training and testing the model

In [7]:
# Create the model
model = Model(inputs=mobilenet.input, outputs=class_outputs)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Load the training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

train_dataset = train_datagen.flow_from_directory(
    '/kaggle/input/road-sign-detection/road_sign_detection/train',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the validation data
val_datagen = ImageDataGenerator(rescale=1./255)

val_dataset = val_datagen.flow_from_directory(
    '/kaggle/input/road-sign-detection/road_sign_detection/val',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'
)

# Train the model
model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

# Evaluate the model on the test data
test_datagen = ImageDataGenerator(rescale=1./255)

test_dataset = test_datagen.flow_from_directory(
    '/kaggle/input/road-sign-detection/road_sign_detection/train',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 4291 images belonging to 7 classes.
Found 622 images belonging to 7 classes.
Epoch 1/300
34/34 [==============================] - 65s 2s/step - loss: 1.9328 - accuracy: 0.2822 - val_loss: 1.6395 - val_accuracy: 0.2846
Epoch 2/300
34/34 [==============================] - 29s 853ms/step - loss: 1.6248 - accuracy: 0.2962 - val_loss: 1.5670 - val_accuracy: 0.2830
Epoch 3/300
34/34 [==============================] - 29s 864ms/step - loss: 1.6012 - accuracy: 0.2892 - val_loss: 1.5401 - val_accuracy: 0.2910
Epoch 4/300
34/34 [==============================] - 29s 848ms/step - loss: 1.5708 - accuracy: 0.3032 - val_loss: 1.5290 - val_accuracy: 0.2878
Epoch 5/300
34/34 [==============================] - 29s 856ms/step - loss: 1.5397 - accuracy: 0.3137 - val_loss: 1.5988 - val_accuracy: 0.2942
Epoch 6/300
34/34 [==============================] - 29s 847ms/step - loss: 1.4670 - accuracy: 0.3423 - val_loss: 1.5833 - val_accuracy: 0.3055
Epoch 7/300
34/34 [==============================] - 29s

#Final testing accuracy

In [8]:
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.metrics import classification_report
import time

test_labels = test_dataset.classes
test_labels = to_categorical(test_labels, num_classes=9)

start_time = time.time()
y_pred = model.predict(test_dataset)
y_pred_bool = np.argmax(y_pred, axis=1)
rounded_labels=np.argmax(test_labels, axis=1)
from sklearn.metrics import classification_report
print(classification_report(y_pred_bool, rounded_labels, digits=4))
print("Time taken to predict the model " + str(time.time() - start_time))

34/34 [==============================] - 22s 624ms/step
              precision    recall  f1-score   support

           0     0.5073    0.1426    0.2226      2181
           1     0.1207    0.1209    0.1208       612
           2     0.0016    0.2500    0.0032         4
           3     0.1289    0.1293    0.1291       611
           4     0.0604    0.1370    0.0838       270
           5     0.0000    0.0000    0.0000         0
           6     0.1582    0.1582    0.1582       613

    accuracy                         0.1396      4291
   macro avg     0.1396    0.1340    0.1025      4291
weighted avg     0.3198    0.1396    0.1766      4291

Time taken to predict the model 22.36392331123352
